{
    "connect_mods": {
        "mqtt": {
            "oringpass": {
                "client_id": "thing:syjmHv7z-",
                "host": "mqtt.paas.oringnet.cloud",
                "port": 1883,
                "username": "syjmHv7z-",
                "userpw": "ZCsC5XNCsk~mzrsy",
                "keepalive": 60,
                "topic": {
                    "Command": "$thing/syjmHv7z-/$cmd/$downlink/",
                    "Downlink config": "$thing/syjmHv7z-/$conf/$expected/",
                    "Uplink config": "$thing/syjmHv7z-/$conf/$reported/",
                    "Data": "$thing/syjmHv7z-/$data/"
                }
            },
            "localdev": {
                "host": "localhost",
                "port": 1883,
                "keepalive": 60
            }
        }
    },
    "request_mods": {
        "container_status_check": {
            "id": "container_status_check",
            "value": {
                "target_image": "nginx:latest"
            }
        

In [3]:
import paho.mqtt.client as mqtt
import logging
import json
import time

class connector_abc():

    def __init__(self):
        pass

    def connect(self):
        """
        this func connects to the server
        """
        raise NotImplementedError


class mqtt_connak(connector_abc):
    
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)
        self.client = mqtt.Client(client_id = self.client_id)
        self.client.on_connect = self.on_connect
        self.client.on_publish = self.on_publish
        
    def __str__(self):
        return "mqtt_connector"
    
    def on_connect(self, client, userdata, flags, rc):
        print("Connected with result code "+str(rc))
        
    def on_publish(self,client,userdata,result):
        print("data published, result " + str(result))
    
    def connect(self):
        if self.username and self.userpw:
            self.client.username_pw_set(self.username, self.userpw)
        try:
            self.client.connect(self.host, self.port, self.keepalive)
        except err as exception:
            print(err)



In [4]:
a = mqtt_connak(**data['mqtt']['oringpass'])

KeyError: 'mqtt'

In [5]:
a.connect()

NameError: name 'a' is not defined

In [6]:
a.client.publish('$thing/syjmHv7z-/$data/request',json.dumps({"id":"temp", "value":"docker"}))

NameError: name 'a' is not defined

In [9]:
import yaml
import json

with open('cloud_publish.yaml') as f:
    
    data = yaml.load(f, Loader=yaml.FullLoader)
    print(json.dumps(data, indent=4))

{
    "connect_mods": {
        "mqtt": {
            "oringpass": {
                "client_id": "thing:syjmHv7z-",
                "host": "mqtt.paas.oringnet.cloud",
                "port": 1883,
                "username": "syjmHv7z-",
                "userpw": "ZCsC5XNCsk~mzrsy",
                "keepalive": 60,
                "topic": {
                    "Command": "$thing/syjmHv7z-/$cmd/$downlink/",
                    "Downlink config": "$thing/syjmHv7z-/$conf/$expected/",
                    "Uplink config": "$thing/syjmHv7z-/$conf/$reported/",
                    "Data": "$thing/syjmHv7z-/$data/"
                },
                "publish_path": {
                    "bucket1": "request"
                }
            },
            "localdev": {
                "host": "localhost",
                "port": 1883,
                "keepalive": 60
            }
        }
    },
    "payload_construct_mods": {
        "container_status_check": {
            "id": "container_sta

In [10]:
import payload_construct_mods as pcmod
import connector_mods as cnmod

f = open('cloud_publish.yaml')
data = yaml.load(f, Loader=yaml.FullLoader)
buff = (data['payload_construct_mods']['container_status_check']['value']).copy()
# data['request_mods']['container_status_check']['value'] = str(buff)
# data['request_mods']['container_status_check']
# mod = container_checker()

In [11]:
a=pcmod.container_checker(**data['payload_construct_mods']['container_status_check'])
a.construct_payload()
a.stringfy_value()
a.payload

{'id': 'container_status_check',
 'value': "{'target_image': 'k8s/gcr.io/pause', 'target_image_tag': 'alp233', 'timestamp': '2020/11/26 16:05:04'}"}

In [12]:
c = cnmod.mqtt_connack(**data['connect_mods']['mqtt']['oringpass'])
c.connect()
k=c.payload_submit(json.dumps(a.payload))
k

connect to mqtt.paas.oringnet.cloud:1883
